In [1]:
import numpy as np
import pandas as pd
import math
import scipy
import sys
from tqdm import tqdm
import functools
import networkx as nx
import itertools

import portion as P
import re
import datetime
import queue

# interval = P.empty()
# for i, j in [(0, 1), (2, 4), (6, 7), (3, 5)]:
#     interval = interval | P.closed(i, j)
# print(interval)

In [17]:
class Blueprint():
    def __init__(self, b_num, ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian) -> None:
        self.number = b_num
        self.ore_ore = ore_ore
        self.clay_ore = clay_ore
        self.obsidian_ore = obsidian_ore
        self.obsidian_clay = obsidian_clay
        self.geode_ore = geode_ore
        self.geode_obsidian = geode_obsidian
        
        self.max_ore = max(ore_ore,clay_ore,obsidian_ore)

    def mine_geodes(self, available_time = 24, initial_state=(0,[0,0,0,0],[1,0,0,0])):
        most_mined_geodes = 0

        states = self.solve(available_time, initial_state)

        tmp_set = []
        max_num = 0
        for s_str in states:
            ss = eval(s_str)
            if ss[1][3] > max_num:
                # print(ss)
                max_num = ss[1][3]
                
        return max_num

    
    def check_state(self,state=(0,[0,0,0,0],[1,0,0,0])):
        time, collected_ores,robots = state
        # print(f'{time=} {collected_ores=} {robots=}')
        # states = set()
        states = []
        
        #check if we can make geode robot
        if collected_ores[0] >= self.geode_ore and collected_ores[2] >= self.geode_obsidian:
            # print('building geode robot')
            tmp_collected_ores = collected_ores.copy()
            tmp_collected_ores[0] -= (self.geode_ore)
            tmp_collected_ores[2] -= (self.geode_obsidian)
            for i,r in enumerate(robots):
                tmp_collected_ores[i] += r
            tmp_robots = robots.copy()
            tmp_robots[3] += 1
            # print(f'new state: {(time+1,tmp_collected_ores,tmp_robots)}')
            # states = states | {str((time+1,tmp_collected_ores,tmp_robots))}
            states.append(str((time+1,tmp_collected_ores,tmp_robots)))
        
        #check if we can make obsidian robot
        if collected_ores[0] >= self.obsidian_ore and collected_ores[1] >= self.obsidian_clay and robots[2] < self.geode_obsidian:
            # print('building obsidian robot')
            tmp_collected_ores = collected_ores.copy()
            tmp_collected_ores[0] -= (self.obsidian_ore)
            tmp_collected_ores[1] -= (self.obsidian_clay)
            for i,r in enumerate(robots):
                tmp_collected_ores[i] += r
            tmp_robots = robots.copy()
            tmp_robots[2] += 1
            # print(f'new state: {(time+1,tmp_collected_ores,tmp_robots)}')
            # states = states | {str((time+1,tmp_collected_ores,tmp_robots))}
            states.append(str((time+1,tmp_collected_ores,tmp_robots)))
        
        #check if we can make clay robot
        if collected_ores[0] >= self.clay_ore and robots[1] < self.obsidian_clay:
            # print('building clay robot')
            tmp_collected_ores = collected_ores.copy()
            tmp_collected_ores[0] -= (self.clay_ore)
            for i,r in enumerate(robots):
                tmp_collected_ores[i] += r
            tmp_robots = robots.copy()
            tmp_robots[1] += 1
            # print(f'new state: {(time+1,tmp_collected_ores,tmp_robots)}')
            # states = states | {str((time+1,tmp_collected_ores,tmp_robots))}
            states.append(str((time+1,tmp_collected_ores,tmp_robots)))

        #check if we can make ore robot
        if collected_ores[0] >= self.ore_ore and robots[0] < self.max_ore:
            # print('building ore robot')
            tmp_collected_ores = collected_ores.copy()
            tmp_collected_ores[0] -= (self.ore_ore)
            for i,r in enumerate(robots):
                tmp_collected_ores[i] += r
            tmp_robots = robots.copy()
            tmp_robots[0] += 1
            # print(f'new state: {(time+1,tmp_collected_ores,tmp_robots)}')
            # states = states | {str((time+1,tmp_collected_ores,tmp_robots))}
            states.append(str((time+1,tmp_collected_ores,tmp_robots)))
        
        missing_resources = any([collected_ores[0] < self.max_ore, (robots[1]>0) and (collected_ores[1] < self.obsidian_clay), (robots[2]>0) and (collected_ores[2] < self.geode_obsidian)])

        #don't build any robot
        for i,r in enumerate(robots):
            collected_ores[i] += r
        
        
        if missing_resources:
            # states = states | {str((time+1,collected_ores,robots))}
            states.append(str((time+1,collected_ores,robots)))
        
        return states
    
    def solve(self, available_time = 24, initial_state=(0, [0, 0, 0, 0], [1, 0, 0, 0])):

        def trim_states(states):
            pass
        
        active_states = {str(initial_state)}
        cnt = True
        max_geode_robots = 0
        while cnt:
            available_time -= 1
            if available_time < 0:
                return active_states
            # print(f'{past_states=}')
            # print(f'{active_states=}')
            # tmp_set = set()
            tmp_set = []

            for state_str in tqdm(active_states):
            # for state_str in active_states:
                state = eval(state_str)
                new_tmp = self.check_state(state)
                for s_str in new_tmp:
                    s = eval(s_str)
                    if s[2][3] > max_geode_robots:
                        max_geode_robots = s[2][3]
                        #we found a solution
                        # print('#we found a solution')
                        # cnt = False
                        # ret_states.append(s)
                # tmp_set = tmp_set | new_tmp
                tmp_set += new_tmp

            active_states = list(set(tmp_set))

            #prune if we have geode robots
            ## removes states where the geode robot count is too low
            if max_geode_robots > 1:
                tmp_set = []
                for state in active_states:
                    if eval(state)[2][3] >= max_geode_robots-1:
                        tmp_set.append(state)
                print(f'remaining time: {available_time}, removed {len(active_states) - len(tmp_set)} states')
                active_states = tmp_set


            ################################
            
            # if len(active_states) > 1:
            #     # prune active set. Keep only the states with the most resources
            #     new_active_set = []
            #     new_discard_set = []

            #     state_dict = {} 
            #     for s in active_states:
            #         state = eval(s)
            #         robots_str = str(state[2])
            #         if robots_str not in state_dict:
            #             state_dict[robots_str] = []
            #         state_dict[robots_str].append((state,s))
            #     for k,v in state_dict.items():
            #         if len(v) > 1:
            #             active = []
            #             discard = []
            #             unknown = []
            #             while len(v) > 1:
            #                 s_1 = v.pop(0)
            #                 state_1, s_str_1 = s_1
            #                 found = False
            #                 for i,s_2 in enumerate(v):

            #                     state_2, s_str_2 = s_2
            #                     ## same robots, compare resources
            #                     s1 = np.array(state_1[1])
            #                     s2 = np.array(state_2[1])
            #                     if np.all(s1 > s2):
            #                         ## s1 is bigger
            #                         active.append(s_str_1)
            #                         discard.append(s_str_2)
            #                         found = True
            #                         break
            #                     if np.all(s1 < s2):
            #                         ## s2 is bigger and we can discard s1
            #                         active.append(s_str_2)
            #                         discard.append(s_str_1)
            #                         found = True
            #                         break
            #                     # ## cannot compare
            #                     # unknown += [s_str_1, s_str_2
            #                 if found:
            #                     v.pop(i)
            #                 else:
            #                     unknown.append(s_str_1)
            #             active += [x[1] for x in v]

            #             active = list(set(active))
            #             discard = list(set(discard))
            #             unknown = list(set(unknown))
            #             for s in unknown:
            #                 if (s not in active) and (s not in discard):
            #                     active.append(s)
            #             new_active_set += active
            #         else:
            #             new_active_set += [x[1] for x in v]
            #     active_states = new_active_set
            ################################
            # elif len(active_states) > 1:
            #     new_active_set = []
            #     new_discard_set = []
            #     new_unknown_set = []
            #     for s_str_1,s_str_2 in itertools.combinations(active_states,2):
            #         if all((any([s_str_1 not in l for l in [new_active_set,new_discard_set]]),any([s_str_2 not in l for l in [new_active_set,new_discard_set]]))):
            #             state_1 = eval(s_str_1)
            #             state_2 = eval(s_str_2)
            #             if state_1[2] == state_2[2]:
            #                 ## same robots, compare resources
            #                 s1 = np.array(state_1[1])
            #                 s2 = np.array(state_2[1])
            #                 if np.all(s1 > s2):
            #                     ## s1 is bigger
            #                     new_active_set.append(s_str_1)
            #                     new_discard_set.append(s_str_2)
            #                     continue
            #                 if np.all(s1 < s2):
            #                     ## s2 is bigger and we can discard s1
            #                     new_active_set.append(s_str_2)
            #                     new_discard_set.append(s_str_1)
            #                     continue
            #             ## cannot compare
            #             new_unknown_set += [s_str_1, s_str_2]
                
            #     new_active_set = list(set(new_active_set))
            #     new_discard_set = list(set(new_discard_set))
            #     new_unknown_set = list(set(new_unknown_set))
            #     for s in new_unknown_set:
            #         if (s not in new_active_set) and (s not in new_discard_set):
            #             new_active_set.append(s)
            #     active_states = new_active_set

            # new_active_set = {}
            # new_discard_set = {}
            # new_unknown_set = {}
            # for s_str_1 in active_states:
            #     state_1 = eval(s_str_1)
            #     state_1_compared = False
            #     for s_str_2 in active_states:
            #         state_2 = eval(s_str_2)
            #         if state_1 == state_2:
            #             continue
            #         if state_1[2] == state_2[2]:
            #             ## same robots, compare resources
            #             s1 = np.array(state_1[1])
            #             s2 = np.array(state_2[1])
            #             if np.all(s1 > s2):
            #                 ## s1 is bigger
            #                 tmp_set.append(s_str_1)
            #                 state_1_compared = True
            #                 break
            #             elif np.all(s1 < s2):
            #                 ## s2 is bigger and we can discard s1
            #                 state_1_compared = True
            #                 break
            #     if not state_1_compared:
            #         tmp_set.append(s_str_1)
            # active_states = new_active_set
                        

                # if str(state[2]) not in best_cases:
                #     best_cases[str(state[2])] = [state[1]]
                # can_drop = False
                # add_to_best_cases = []
                # for s in best_cases[str(state[2])]:
                #     if np.all(np.array(state[1]) < np.array(s[1])) and np.all(np.array(state[1]) > np.array(s[1])) == False:
                #         pass
                #     elif np.all(np.array(state[1]) > np.array(s[1])) and np.all(np.array(state[1]) < np.array(s[1])) == False:
                #         add_to_best_cases.append()

        return None


In [33]:
file1 = open('19.txt', 'r')
lines = file1.readlines()

blueprints = []

for l in lines:
    b_num, ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian = [int(i) for i in re.findall('\d+',l.replace('\n',''))]
    blueprints.append(Blueprint(b_num, ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian))

    
    # print(x,y,z)

print(f'loaded {len(blueprints)} blueprints')



loaded 30 blueprints


In [41]:
# b = blueprints[0]

# b.mine_geodes()
# # print(f'{len(t)}')
# # t

In [39]:
ans1 = []
for i,b in enumerate(blueprints):
    tmp = b.mine_geodes()
    print(i+1,tmp)
    ans1.append((i+1)*tmp)
ans1

100%|██████████| 57082/57082 [00:04<00:00, 13489.89it/s]


1 0


100%|██████████| 181854/181854 [00:14<00:00, 12588.04it/s]


2 2


100%|██████████| 273564/273564 [00:23<00:00, 11794.52it/s]


...

100%|██████████| 134186/134186 [00:10<00:00, 12543.24it/s]


29 0


100%|██████████| 588367/588367 [00:50<00:00, 11616.82it/s]


30 5


[0,
 4,
 9,
 4,
 0,
 0,
 14,
 0,
 9,
 50,
 0,
 0,
 0,
 42,
 210,
 80,
 17,
 18,
 19,
 100,
 84,
 330,
 0,
 48,
 0,
 26,
 243,
 224,
 0,
 150]

In [40]:
sum(ans1)

1681

# Second part

In [18]:
file1 = open('19.txt', 'r')
lines = file1.readlines()

blueprints = []

for l in lines:
    b_num, ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian = [int(i) for i in re.findall('\d+',l.replace('\n',''))]
    blueprints.append(Blueprint(b_num, ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian))

    
    # print(x,y,z)

print(f'loaded {len(blueprints)} blueprints')



loaded 3 blueprints


In [19]:
ans2 = []
for i,b in enumerate(blueprints):
    tmp = b.mine_geodes(available_time=32)
    print(i+1,tmp)
    ans2.append(tmp)
ans2

100%|██████████| 2463526/2463526 [02:29<00:00, 16455.82it/s]


remaining time: 1, removed 3172372 states


100%|██████████| 1023342/1023342 [01:01<00:00, 16532.37it/s]


remaining time: 0, removed 0 states
1 6


100%|██████████| 181854/181854 [00:10<00:00, 16842.25it/s]


...

100%|██████████| 294332/294332 [00:18<00:00, 16073.48it/s]


remaining time: 1, removed 582091 states


100%|██████████| 116820/116820 [00:07<00:00, 16493.50it/s]


remaining time: 0, removed 271615 states
3 29


[6, 31, 29]

In [20]:
prod = 1
for x in ans2:
    prod *= x
prod

5394